In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install rdkit

In [20]:
!pip install mordred

In [21]:
!pip install pubchempy

In [22]:
import pandas as pd
import numpy as np
from rdkit import Chem
from mordred import Calculator, descriptors
import pubchempy as pcp
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import plotly.express as px
from scipy import stats
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import plotly.graph_objects as go


In [23]:
mordredset = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Alghoritms_and_Data/mordredset.csv', low_memory=False)
mordredset

,Unnamed: 0,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,7.099577,7.101505,0,0,11.822996,2.394704,4.468530,11.822996,1.313666,...,9.030137,54.421098,124.027277,9.540560,77,9,48.0,57.0,2.5833333333333335,1.972222
1,1,7.662296,7.839488,0,1,11.506971,2.761148,5.075449,11.506971,1.278552,...,10.358473,64.099976,123.068414,6.837134,78,9,60.0,82.0,2.3958333333333335,1.819444
2,2,6.990188,6.993729,0,0,11.351628,2.439897,4.731489,11.351628,1.261292,...,9.399389,53.862370,126.068080,6.635162,82,9,48.0,58.0,2.5833333333333335,2.055556
3,3,8.242641,7.838188,0,0,12.692477,2.748777,5.083300,12.692477,1.410275,...,10.316094,64.041419,121.052764,7.565798,68,8,66.0,90.0,1.4166666666666665,1.666667
4,4,7.232561,7.325997,0,0,9.982031,2.715195,5.430389,9.982031,1.109115,...,10.682377,42.876740,124.052429,7.297202,78,10,56.0,74.0,3.986111111111111,1.854167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,7.099577,7.101505,0,0,11.822996,2.394704,4.468530,11.822996,1.313666,...,9.030137,54.421098,122.036779,8.135785,77,9,48.0,57.0,2.5833333333333335,1.972222
19996,19996,5.794754,6.405580,0,2,9.845163,2.392005,4.379696,9.845163,1.230645,...,8.885579,56.024874,109.063997,7.270933,66,7,38.0,44.0,3.8333333333333335,1.888889
19997,19997,6.542301,6.735922,0,0,10.778133,2.300436,4.354483,10.778133,1.197570,...,8.720787,51.566550,129.042593,8.065162,85,9,42.0,47.0,4.083333333333334,2.111111
19998,19998,6.459522,6.839406,0,0,10.140867,2.620462,4.931919,10.140867,1.267608,...,9.889845,60.114861,116.047344,7.252959,58,7,48.0,62.0,3.0347222222222223,1.666667


In [24]:
RdKitSet = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Alghoritms_and_Data/RdKitSet.csv', low_memory=False)
RdKitSet

,Unnamed: 0,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,7.319537,7.319537,0.078704,-2.174306,0.548360,31.777778,124.099,120.067,124.027277,...,0,0,0,0,0,0,0,0,0,0
1,1,11.814583,11.814583,0.510579,-2.814931,0.427978,105.555556,123.155,114.083,123.068414,...,0,0,0,0,0,0,0,0,0,0
2,2,11.343194,11.343194,1.960486,-3.424421,0.481048,105.333333,126.155,116.075,126.068080,...,0,0,0,0,0,0,0,0,0,0
3,3,8.063079,8.063079,0.225231,-2.546875,0.455175,128.000000,121.139,114.083,121.052764,...,0,0,0,0,0,0,0,0,0,0
4,4,7.669444,7.669444,1.620417,-2.954167,0.457264,88.444444,124.139,116.075,124.052429,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,11.489745,11.489745,0.449074,-2.731481,0.520376,37.111111,122.123,116.075,122.036779,...,0,0,0,0,0,0,0,0,0,0
19996,19996,8.490484,8.490484,0.375000,-2.913403,0.274077,61.625000,109.132,102.076,109.063997,...,0,0,0,0,0,0,0,0,0,0
19997,19997,11.169792,11.169792,1.003009,-3.147225,0.503767,57.555556,129.115,122.059,129.042593,...,0,0,0,0,0,0,0,0,0,0
19998,19998,7.560000,7.560000,2.416667,-2.743750,0.421262,106.000000,116.116,108.052,116.047344,...,0,0,0,0,0,0,0,0,0,0


Объединяем все доступные Датасеты

In [25]:
# Выполняем объединение DataFrame
New_data = pd.concat([df, mordredset, RdKitSet], axis=1)



In [26]:
#Information on the table
New_data.info(verbose = True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2056 columns):
 #     Column                    Non-Null Count  Dtype  
---    ------                    --------------  -----  
 0     Unnamed: 0                20000 non-null  int64  
 1     mol_id                    20000 non-null  object 
 2     smiles                    20000 non-null  object 
 3     A                         20000 non-null  float64
 4     B                         20000 non-null  float64
 5     C                         20000 non-null  float64
 6     mu                        20000 non-null  float64
 7     alpha                     20000 non-null  float64
 8     homo                      20000 non-null  float64
 9     lumo                      20000 non-null  float64
 10    gap                       20000 non-null  float64
 11    r2                        20000 non-null  float64
 12    zpve                      20000 non-null  float64
 13    u0                        2

В датасете нет нанов

In [27]:
New_data = New_data.drop(columns = ['mol_id', 'Unnamed: 0'])
New_data

,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,N=C1OCC2=C1ON=C2,3.42254,1.67692,1.13352,4.5643,67.77,-0.2620,-0.0572,0.2048,972.8206,...,0,0,0,0,0,0,0,0,0,0
1,O=C1CCC11C2CCN12,3.00351,1.58602,1.24538,1.9055,77.52,-0.2402,-0.0312,0.2091,1039.1092,...,0,0,0,0,0,0,0,0,0,0
2,O=CC1CCC2OCC12,3.11416,1.26756,1.03765,1.6498,75.15,-0.2483,-0.0267,0.2215,1166.1361,...,0,0,0,0,0,0,0,0,0,0
3,C1C2C3N=C4OC1C4C23,3.12126,2.32028,1.94917,3.6237,69.65,-0.2277,-0.0211,0.2066,763.5215,...,0,0,0,0,0,0,0,0,0,0
4,CC1(O)C2CC1(O2)C#C,2.37047,1.54170,1.35971,1.7895,76.02,-0.2495,0.0074,0.2569,1039.3059,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,c1coc2c1C(=O)CC2,3.17605,1.63047,1.09187,3.0955,73.12,-0.2383,-0.0256,0.2127,1033.0148,...,0,0,0,0,0,0,0,0,0,0
19996,CC1CN1C(=N)C#N,3.12217,1.77668,1.34512,3.1791,69.13,-0.2612,-0.0537,0.2075,927.9641,...,0,0,0,0,0,0,0,0,0,0
19997,COC1CC(=N)OC1=O,3.40512,1.21702,0.92748,2.1467,67.72,-0.2631,-0.0214,0.2416,1215.9934,...,0,0,0,0,0,0,0,0,0,0
19998,OC1CC11OCC1O,3.22232,1.74690,1.53428,1.4181,60.90,-0.2393,0.0544,0.2937,882.3828,...,0,0,0,0,0,0,0,0,0,0


In [28]:
New_data.isnull().sum() # Подсчет нанов


smiles              0
A                   0
B                   0
C                   0
mu                  0
                   ..
fr_thiazole         0
fr_thiocyan         0
fr_thiophene        0
fr_unbrch_alkane    0
fr_urea             0
Length: 2052, dtype: int64

Let's consider whether there are molecules in the dataset whose solubility and descriptors are represented several times. To do this, let's count the number of repetitions of SMILES.

Теперь закодируем столбец Smiles для того чтобы тот не потерялся в процессе чистки object, потому что сейчас smiles это object

In [29]:
ord_enc = OrdinalEncoder()
New_data["smiles_code"] = ord_enc.fit_transform(New_data[["smiles"]])
New_data.head(11)

,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,smiles_code
0,N=C1OCC2=C1ON=C2,3.42254,1.67692,1.13352,4.5643,67.77,-0.2620,-0.0572,0.2048,972.8206,...,0,0,0,0,0,0,0,0,0,14905.0
1,O=C1CCC11C2CCN12,3.00351,1.58602,1.24538,1.9055,77.52,-0.2402,-0.0312,0.2091,1039.1092,...,0,0,0,0,0,0,0,0,0,15889.0
2,O=CC1CCC2OCC12,3.11416,1.26756,1.03765,1.6498,75.15,-0.2483,-0.0267,0.2215,1166.1361,...,0,0,0,0,0,0,0,0,0,16518.0
3,C1C2C3N=C4OC1C4C23,3.12126,2.32028,1.94917,3.6237,69.65,-0.2277,-0.0211,0.2066,763.5215,...,0,0,0,0,0,0,0,0,0,957.0
4,CC1(O)C2CC1(O2)C#C,2.37047,1.54170,1.35971,1.7895,76.02,-0.2495,0.0074,0.2569,1039.3059,...,0,0,1,0,0,0,0,0,0,4821.0
5,CC1=CN(C(=O)OC1)C,2.98866,1.36443,0.96322,4.4884,77.50,-0.2127,0.0040,0.2167,1218.6691,...,0,0,0,0,0,0,0,0,0,6537.0
6,CN1CC11COC=NC1,2.85410,1.46427,1.24798,2.7784,76.24,-0.2279,0.0265,0.2544,1075.5431,...,0,0,0,0,0,0,0,0,0,11767.0
7,CC1CC(CCC#C)O1,4.27034,0.69008,0.66468,1.9752,85.68,-0.2459,0.0604,0.3064,1761.8615,...,0,0,1,0,0,0,0,0,0,7592.0
8,N=C1OCC(=O)C11CO1,2.31264,2.01810,1.16837,2.5131,64.30,-0.2626,-0.0665,0.1961,986.3012,...,0,0,0,0,0,0,0,0,0,14875.0
9,CC(C)(O)C(C#C)C#N,1.83037,1.63257,1.07635,4.6341,77.35,-0.2832,0.0133,0.2965,1178.3146,...,0,0,1,0,0,0,0,0,0,3148.0


In [30]:
column_to_move = New_data.pop('smiles_code')

# Вставляем столбец в начало DataFrame
New_data.insert(0, 'smiles_code', column_to_move)

<ipython-input-30-69bc927c8cb8>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  New_data.insert(0, 'smiles_code', column_to_move)


In [31]:
del New_data['smiles']
New_data.head(11)


,smiles_code,A,B,C,mu,alpha,homo,lumo,gap,r2,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,14905.0,3.42254,1.67692,1.13352,4.5643,67.77,-0.2620,-0.0572,0.2048,972.8206,...,0,0,0,0,0,0,0,0,0,0
1,15889.0,3.00351,1.58602,1.24538,1.9055,77.52,-0.2402,-0.0312,0.2091,1039.1092,...,0,0,0,0,0,0,0,0,0,0
2,16518.0,3.11416,1.26756,1.03765,1.6498,75.15,-0.2483,-0.0267,0.2215,1166.1361,...,0,0,0,0,0,0,0,0,0,0
3,957.0,3.12126,2.32028,1.94917,3.6237,69.65,-0.2277,-0.0211,0.2066,763.5215,...,0,0,0,0,0,0,0,0,0,0
4,4821.0,2.37047,1.54170,1.35971,1.7895,76.02,-0.2495,0.0074,0.2569,1039.3059,...,0,0,0,1,0,0,0,0,0,0
5,6537.0,2.98866,1.36443,0.96322,4.4884,77.50,-0.2127,0.0040,0.2167,1218.6691,...,0,0,0,0,0,0,0,0,0,0
6,11767.0,2.85410,1.46427,1.24798,2.7784,76.24,-0.2279,0.0265,0.2544,1075.5431,...,0,0,0,0,0,0,0,0,0,0
7,7592.0,4.27034,0.69008,0.66468,1.9752,85.68,-0.2459,0.0604,0.3064,1761.8615,...,0,0,0,1,0,0,0,0,0,0
8,14875.0,2.31264,2.01810,1.16837,2.5131,64.30,-0.2626,-0.0665,0.1961,986.3012,...,0,0,0,0,0,0,0,0,0,0
9,3148.0,1.83037,1.63257,1.07635,4.6341,77.35,-0.2832,0.0133,0.2965,1178.3146,...,0,0,0,1,0,0,0,0,0,0


In [32]:
New_data.to_csv('/content/drive/MyDrive/Colab_Notebooks/Alghoritms_and_Data/New_data1.csv')